# Introduction

The reference for matrix computations in CUDA is the cuBLAS library.
Similar to other basic linear algebra (BLAS) libraries, cuBLAS does not
have a particular data structure for representing matrics. Instead, the data
in the matrix is stored in a 1D array, and it is up to the user to keep track
of matrix dimensions, memory allocation, etc.
In our experience, this is very error prone and coders end up spending hours
hunting for bugs that are often just typos. This is one of the reasons that frameworks like PyTorch and Tensorflow provide their own tensor data structures.
We do the same, but keep the data structure as simple as possible so that you can understand easily what's under the hood.

We propose a __simple cuBLAS-compatible data structure__, called `fmatrix`, for representing matrices
with single precision, and provide a collection of __functions for routine tasks__ such as
- easy access to matrix elements (no need to compute element addresses),
- creating matrices (allocating memory) on the host and the device, including matrices with random values,
- copying matrices between host and device,
- printing matrices and reading/writing with CSV files.

We highly encourage you to look at how these functions are coded, and then use them in their own code rather than directly working with the 1D array data. This will make their code faster to develop, more readable and help avoid bugs. This is not just about understanding how the `fmatrix` code works -- the same principles are at work in __all__ the deep learning frameworks.

In this notebook, we first show the `fmatrix` structure and accompanying functions, then give examples of matrix functions implemented with `fmatrix`, and finally show different ways to test that these functions work as intended.
For exhaustive, automated testing, we generate and check test cases with Numpy. Comparing the output of the GPU code with established software should greatly increase the confidence. The same principle could be applied to compare with the output of high-level frameworks like PyTorch or Tensorflow.



# Installation

Check that the NVIDIA compiler is present.

In [ ]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


Download the cuda-samples repository to get header files we'll use:

In [ ]:
!git clone https://github.com/NVIDIA/cuda-samples.git

Cloning into 'cuda-samples'...
remote: Enumerating objects: 14775, done.
remote: Counting objects: 100% (4167/4167), done.
remote: Compressing objects: 100% (710/710), done.
remote: Total 14775 (delta 3687), reused 3733 (delta 3454), pack-reused 10608
Receiving objects: 100% (14775/14775), 132.87 MiB | 14.44 MiB/s, done.
Resolving deltas: 100% (12456/12456), done.
Updating files: 100% (3941/3941), done.


## CUDA Utilities

In [1]:
%%writefile cuda_stuff.cuh
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <helper_cuda.h>

#ifndef cuda_stuff_H
#define cuda_stuff_H

/* transform matrix index to vector offset
   Since CUDA uses column major,
   nb_rows = number of rows */
#define IDX2C(i,j,nb_rows) (((j)*(nb_rows))+(i))

//MACRO TO DEBUGG CUDA FUNCTIONS
/** Error checking,
 *  taken from https://stackoverflow.com/questions/14038589/what-is-the-canonical-way-to-check-for-errors-using-the-cuda-runtime-api
 */
#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess)
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}
/** Error checking for use with CUDA Dynamic Parallelism */
/*
#define cdpErrchk(ans) { cdpAssert((ans), __FILE__, __LINE__); }
__device__ void cdpAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess)
   {
      printf("GPU kernel assert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) assert(0);
   }
}
*/
void device_synchronize();

#endif


Writing cuda_stuff.cuh


In [2]:
%%writefile cuda_stuff.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <helper_cuda.h>
#include "cuda_stuff.cuh"

void device_synchronize(){
    gpuErrchk(cudaDeviceSynchronize());
}

Writing cuda_stuff.cu


# The fmatrix Data Structure

Similar to cuBLAS, a 2D matrix is represented by a 1D array that stores one column of the matrix after another (this is called _linearization_).
The `fmatrix` structure contains a `data` pointer to such an array, which is 100% compatible with cuBLAS. In addition, it stores the number of columns and rows in the matrix.

In [3]:
%%writefile fmatrix.cuh
#ifndef fmatrices_H
#define fmatrices_H
#include "cuda_stuff.cuh" // for IDX2C

////////////////////////////////////////
// basic data structure and access macro
////////////////////////////////////////
typedef struct {
    float* data;
    int cols;
    int rows;
} fmatrix;

/** Access element (i,j) of matrix M
 *
 *  Usage example:
 *  For computing A = B^T + C), loop over i and j with:
 *    getfm(A,i,j) = getfm(B,j,i) + getfm(C,i,j);
 **/
#define getfm(M,i,j) (M.data[IDX2C(i,j,M.rows)])

////////////////////////////////////////
// utility functions
////////////////////////////////////////
/** Returns the number of elements in the matrix.
 *
 *  Useful for computing, e.g., the size
 *  of a 1D-vector that contains the same numbers.
 */
 __host__
 __device__
int fmatrix_elements(fmatrix mat);

/** Returns the memory occupied by the matrix elements in bytes
 *  (not including the variables in the struct mat).
 *
 *  Useful for allocating memory for the data.
 */
 __host__
 __device__
int fmatrix_size(fmatrix mat);

/** Assert that the matrix is coherent: all fields nonzero. */
 __host__
 __device__
void fmatrix_assert(fmatrix mat);

////////////////////////////////////////
// Create, copy, destroy
////////////////////////////////////////
/** Allocate memory on host */
fmatrix fmatrix_create_on_host(int rows, int cols);

/** Allocate memory on device */
fmatrix fmatrix_create_on_device(int rows, int cols);

/** Create a matrix representing columns [a,b) of M.
 *  Note that the new matrix uses a pointer to the
 *  data of M. The data is not copied to a new location.
 *  If M is destroyed, this matrix is useless.
 */
fmatrix fmatrix_subcolumns(fmatrix M, int a, int b);

/** Copy data from matrix on device to host
 *  (no memory allocation). */
void fmatrix_data_to_host(fmatrix mat_host, fmatrix mat_device);

/** Copy data from matrix on host to device
 *  (no memory allocation). */
void fmatrix_data_to_device(fmatrix mat_host, fmatrix mat_device);

/** Copy matrix from device to host, allocating new memory. */
fmatrix fmatrix_copy_to_host(fmatrix mat_device);

/** Copy matrix from host to device, allocating new memory. */
fmatrix fmatrix_copy_to_device(fmatrix mat_host);

/** Free data memory on host.
 *  This zeros out the data pointer of the fmatrix struct,
 *  so a pointer is required. */
void fmatrix_free_on_host(fmatrix* mat);

/** Free data memory on device.
 *  This zeros out the data pointer of the fmatrix struct,
 *  so a pointer is required. */
void fmatrix_free_on_device(fmatrix* mat);

////////////////////////////////////////
// Input and Output
////////////////////////////////////////

/** Print the first nb rows of the matrix mat
 *  on the host.
 *  If nb<0, print all rows.
 */
 __host__
 __device__
void fmatrix_print(fmatrix mat, int nb=-1);

/** Print the first nb rows of the matrix mat
 *  on the device.
 *  If nb<0, print all rows.
 *
 *  This version copies the matrix to host first.
 */
void fmatrix_device_print(fmatrix mat, int nb=-1);

/** Print a matrix to a csv file.
 *
 *  This version copies the matrix to host first.
 */
void fmatrix_device_to_csv(const char* filename, fmatrix mat);

/** Read a matrix from a csv file.
 *
 *  This version creates the matrix on the host first.
 */
fmatrix fmatrix_device_from_csv(const char* filename);

////////////////////////////////////////
// Useful
////////////////////////////////////////

/** Create a matrix with random values between -1 and 1
 *  on the device */
fmatrix fmatrix_create_random_on_device(int rows, int cols);

#endif


Writing fmatrix.cuh


In [4]:
%%writefile fmatrix.cu
#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include <helper_cuda.h>
#include <curand.h>
#include <curand_kernel.h>
#include "cuda_stuff.cuh"
#include "fmatrix.cuh"

// for reading CSV files, we use some C++
#include <iostream>
#include <iomanip>
#include <fstream>
#include <string>

int fmatrix_elements(fmatrix mat) {
     return mat.cols*mat.rows;
}

int fmatrix_size(fmatrix mat) {
//    fmatrix_assert(mat);
     return fmatrix_elements(mat) * sizeof(float);
}

void fmatrix_assert(fmatrix mat) {
    assert(mat.data);
    assert(mat.cols);
    assert(mat.rows);
}

fmatrix fmatrix_create_on_host(int rows, int cols) {
    assert(cols>0);
    assert(rows>0);
    fmatrix mat;
    mat.cols = cols;
    mat.rows = rows;
    mat.data = (float*)malloc(fmatrix_size(mat));
    assert(mat.data);
    return mat;
}

fmatrix fmatrix_create_on_device(int rows, int cols) {
    assert(cols>0);
    assert(rows>0);
    fmatrix mat;
    mat.cols = cols;
    mat.rows = rows;
    gpuErrchk(
        cudaMalloc((void **)&(mat.data), fmatrix_size(mat))
    );
    return mat;
}

void fmatrix_data_to_device(fmatrix mat_host, fmatrix mat_device) {
    fmatrix_assert(mat_host);
    fmatrix_assert(mat_device);
    assert(mat_host.cols==mat_device.cols);
    assert(mat_host.rows==mat_device.rows);
    gpuErrchk(
        cudaMemcpy( mat_device.data, mat_host.data,
                   fmatrix_size(mat_host),
                   cudaMemcpyHostToDevice
                   )
        );
}

void fmatrix_data_to_host(fmatrix mat_host, fmatrix mat_device) {
    fmatrix_assert(mat_host);
    fmatrix_assert(mat_device);
    assert(mat_host.cols==mat_device.cols);
    assert(mat_host.rows==mat_device.rows);
    gpuErrchk(
        cudaMemcpy( mat_host.data, mat_device.data,
                   fmatrix_size(mat_device),
                   cudaMemcpyDeviceToHost
                   )
        );
}

fmatrix fmatrix_copy_to_host(fmatrix mat_device) {
    fmatrix_assert(mat_device);
    fmatrix mat_host = fmatrix_create_on_host(mat_device.rows, mat_device.cols);
    fmatrix_data_to_host(mat_host,mat_device);
    return mat_host;
}

fmatrix fmatrix_copy_to_device(fmatrix mat_host) {
    fmatrix_assert(mat_host);
    fmatrix mat_device = fmatrix_create_on_device(mat_host.rows, mat_host.cols);
    fmatrix_data_to_device(mat_host,mat_device);
    return mat_device;
}

/** We could do it like this, but it would not set our pointer M.data to 0.
... fmatrix_free_on_host(M)
void fmatrix_free_on_host(fmatrix mat) {
    fmatrix_assert(mat);
  free(mat.data);
  mat.data = 0;
  mat.cols = 0;
  mat.rows = 0;
}
*/

void fmatrix_free_on_host(fmatrix* mat) {
    fmatrix_assert(*mat);
  free(mat->data);
  mat->data = 0;
  mat->cols = 0;
  mat->rows = 0;
}

void fmatrix_free_on_device(fmatrix* mat) {
    fmatrix_assert(*mat);
  gpuErrchk(cudaFree(mat->data));
  mat->data = 0;
  mat->cols = 0;
  mat->rows = 0;
}

fmatrix fmatrix_subcolumns(fmatrix M, int a, int b) {
    fmatrix_assert(M);
    fmatrix A = {
        .data = &getfm(M,0,a),
        .cols = b-a,
        .rows = M.rows
    };
    fmatrix_assert(A);
    return A;
}


__host__
__device__
void fmatrix_print(fmatrix mat, int nb){
    if (nb<0 || nb > mat.rows) {
        nb = mat.rows;
    }
    printf("[\n");
    for (int i = 0 ; i < nb; i++){
      for (int j = 0 ; j<mat.cols; j++){
        printf("%f", getfm(mat,i,j));
        if (j+1<mat.cols) {
          printf(",\t");
        }
      }
      if (i+1<nb) {
        printf(";\n");
      }
    }
    if (nb < mat.rows) {
      printf("\n...\n");
    }
  printf("\n]\n");
}

void fmatrix_device_print(fmatrix mat, int nb){
   // allocate copy
   fmatrix tmp = fmatrix_copy_to_host(mat);
   fmatrix_print(tmp,nb);
   fmatrix_free_on_host(&tmp);
}

void fmatrix_device_to_csv(const char* filename, fmatrix mat) {
  // Open file
  FILE* fp = fopen(filename, "w");
  // allocate copy
  fmatrix tmp = fmatrix_copy_to_host(mat);
  for (int i = 0 ; i < tmp.rows; i++){
    for (int j = 0 ; j<tmp.cols; j++){
      // Note: %.15g gives 15 significant digits (full double precision)
      fprintf(fp,"%.15g", getfm(tmp,i,j));
      if (j+1<tmp.cols) {
        fprintf(fp,",");
      }
    }
    fprintf(fp,"\n");
  }
  fmatrix_free_on_host(&tmp);
  // Close file
  fclose(fp);
}

__global__
void fmatrix_create_random_on_device_kernel(fmatrix M) {
    // choose a seed (here: the same each launch)
    unsigned long seed = 0;
    int sequence = 0;
    // first, initialize the random numbers
    curandState state;
    curand_init(seed, sequence, 0, &state);
    for (int i = 0; i < fmatrix_elements(M); ++i) {
        // curand_uniform creates numbers between 0 and 1
        M.data[i] = (curand_uniform(&state)-0.5)*2.0;
    }
}

fmatrix fmatrix_create_random_on_device(int rows, int cols) {
    // Create an uninitialized matrix on the device
    fmatrix M = fmatrix_create_on_device(rows,cols);
    // Call a kernel with a single thread to fill the values
    fmatrix_create_random_on_device_kernel<<<1,1>>>(M);

    return M;
}

/* Count the number of rows and columns in a csv files (without headers) */
void count_elements_in_csv(const char* filename, int* rows, int* cols) {
  // Note: for the sake of convenience, we use some C++ functions here
  using namespace std;

  *rows = 0;
  *cols = 0;
  string row_as_string;
  string value;
  ifstream infile;
  infile.open(filename, ifstream::in);
	if (infile.is_open())
  {
    while (getline(infile, row_as_string, '\n')) {
				istringstream line_stream(row_as_string);
        int tempcols = 0;
        while (getline(line_stream, value, ',')) {
          ++tempcols;
        }
        if (tempcols > *cols) {
           *cols = tempcols;
        }
        ++(*rows);
			}
		infile.close();
	}
	else cout << "Cannot open file." << endl;
}

/** Read the data from a csv file into an fmatrix on the host.
 *  Careful: We assume that the matrix has the right dimensions!
 *  Use count_elements_in_csv(...) to get the dimensions if
 *  unknown.
 */
void fmatrix_fill_from_csv(fmatrix h_M,const char* filename) {
  // Note: for the sake of convenience, we use some C++ functions here
  using namespace std;
  string row_as_string;
  string value;
  ifstream infile;
  infile.open(filename, ifstream::in);
  int row = 0;
	if (infile.is_open())
  {
    while (getline(infile, row_as_string, '\n')) {
				istringstream line_stream(row_as_string);
        int col = 0;
        while (getline(line_stream, value, ',')) {
					getfm(h_M,row,col) = strtod(value.c_str(), NULL);
          ++col;
				}
        ++row;
			}
		infile.close();
	}
	else cout << "Cannot open file." << endl;
}

fmatrix fmatrix_device_from_csv(const char* filename) {
  // first read the file to count the number of elements
  int rows = 0;
  int cols = 0;
  count_elements_in_csv(filename,&rows,&cols);

  // allocate the matrix on the host
  fmatrix h_M = fmatrix_create_on_host(rows,cols);

  // read the data into the host matrix
  fmatrix_fill_from_csv(h_M,filename);

  // copy the matrix to the device
  fmatrix M = fmatrix_copy_to_device(h_M);

  // destroy the host matrix
  fmatrix_free_on_host(&h_M);

  return M;
}




Writing fmatrix.cu


# Writing Matrix Functions with fmatrix
We will look at two examples that illustrate how to create, manipulate and compute using the fmatrix structure.
The first example is the matrix transpose, which generates a new matrix.
The second example is the matrix multiplication. It overwrites the data of an existing matrix with the result of the multiplication.

We start by looking at the code. Testing the code will be discussed in the section that comes afterwards.

## Matrix Addition
Our first example is to add one matrix to another, multiplied by a scalar.
We need to functions: One is a normal C function that is called on the host, the other is the kernel, whose code will be executed on the GPU. In this example, the host function doesn't do much more than launch the kernel.
Have a look at the error checks, which ensure that both matrices are compatible.

In [5]:
%%writefile fmatrix_add.cu
#include "fmatrix.cuh"
#include <assert.h>

#define THREADS_PER_BLOCK 1024
__global__
void fmatrix_add_kernel(fmatrix P,float a,fmatrix Y) {
    int idx = blockIdx.x*blockDim.x+threadIdx.x;
    int j = idx / P.rows;
    int i = idx % P.rows;
    if (i < P.rows && j < P.cols ){
        getfm(P,i,j) += a*getfm(Y,i,j);
    }
}

/** Compute P = P + a*Y */
void fmatrix_add(fmatrix P,float a,fmatrix Y) {
    fmatrix_assert(P);
    fmatrix_assert(Y);
    assert(P.rows == Y.rows);
    assert(P.cols == Y.cols);
    int threadsPerBlock = fmatrix_elements(P);
    int blocksPerGrid = 1;
    if (threadsPerBlock > THREADS_PER_BLOCK){
        blocksPerGrid = (threadsPerBlock-1)/THREADS_PER_BLOCK+1;
        threadsPerBlock = THREADS_PER_BLOCK;
    }
    fmatrix_add_kernel<<< blocksPerGrid, threadsPerBlock >>>(P,a,Y);
    gpuErrchk( cudaPeekAtLastError() );
}

Writing fmatrix_add.cu


Note that the implementation for add doesn't include any synchronization. This is so that you can launch adds in parallel, which is perfectly fine if the date for both tasks is independent. Here's a small example:
```
// Without waiting:
fmatrix_add(P1,a,Y1);
fmatrix_add(P2,a,Y2);
deviceSynchronize();

// Waiting between each
fmatrix_add(P1,a,Y1);
deviceSynchronize();
fmatrix_add(P2,a,Y2);
deviceSynchronize();
```

## Matrix Multiplication

Here, we will create a new matrix A that contains the product of two matrices B and C.


In [6]:
%%writefile fmatrix_mult.cu
#include "fmatrix.cuh"
#include <assert.h>

#define THREADS_PER_BLOCK 1024

__global__
void fmatrix_multiplication_kernel(fmatrix A, float f, fmatrix B, fmatrix C) {
    // Each thread multiplies one row of B with one column of C
    int idx = blockIdx.x*blockDim.x+threadIdx.x;
    int j = idx / A.rows;
    int i = idx % A.rows;
    if (i < A.rows && j < A.cols ){
        getfm(A,i,j) = 0.0;
        for (int k = 0; k < B.cols; ++k) {
          getfm(A,i,j) += f*getfm(B,i,k)*getfm(C,k,j);
        }
    }
}

/* Compute A = f*B*C */
void fmatrix_mult(fmatrix A, float f, fmatrix B, fmatrix C) {
    // First let's check for errors in the argument M.
    // This can help a LOT when debugging.
    // A,B,C need to have nonzero pointers etc.
    // fmatrix_assert(A);
    // fmatrix_assert(B);
    // fmatrix_assert(C);
    assert(A.rows == B.rows);
    assert(A.cols == C.cols);
    assert(B.cols == C.rows);

    // take one thread per element, and distribute
    // over as many blocks as necessary given
    // the hardware limit on the number of threads per block
    int threadsPerBlock = fmatrix_elements(A);
    int blocksPerGrid = 1;
    if (threadsPerBlock > THREADS_PER_BLOCK){
        blocksPerGrid = (threadsPerBlock-1)/THREADS_PER_BLOCK+1;
        threadsPerBlock = THREADS_PER_BLOCK;
    }
    fmatrix_multiplication_kernel<<< blocksPerGrid, threadsPerBlock >>>(A,f,B,C);
    // check for errors
    gpuErrchk( cudaPeekAtLastError() );
    // wait for the kernel to finish
    device_synchronize();
}



Writing fmatrix_mult.cu


# Testing the Matrix Functions

According to the literature, 50% to 75% percent of development time is dedicated to testing, verifying and validating code. This percentage refers to professional software developers, who know how to set up tests. If you don't test your functions individually before using them in larger pieces of code, you can easily end up spending 95% to 99% of your time debugging.

In the next sections, we will look at a few techniques to quickly set up tests, first for manual, visual inspection, and then for automatic testing.

## Visual Inspection

The following test code calls the add function on a random matrix and prints the result. Since the resulting values are easy to check manually, we can confirm that our function works simply by looking at the output.

In [7]:
%%writefile test.cu
#include "fmatrix.cuh"
#include "fmatrix_add.cu"

int main() {
    // create a random matrix
    fmatrix M = fmatrix_create_random_on_device(3,5);
    printf("original matrix:\n");
    fmatrix_device_print(M);
    // add the matrix to itself
    fmatrix_add(M,1.0,M);
    // print
    printf("matrix added to itself:\n");
    fmatrix_device_print(M);
    // free the memory
    fmatrix_free_on_device(&M);
}

Writing test.cu


In [ ]:
!nvcc -arch=sm_35 -Wno-deprecated-gpu-targets -g -G -I cuda-samples/Common/ -L/usr/local/cuda/include test.cu fmatrix.cu cuda_stuff.cu

In [ ]:
!./a.out

original matrix:
[
0.480439,	-0.686023,	-0.544713,	-0.624402,	0.949322;
-0.123098,	-0.857265,	-0.341283,	0.830767,	0.094939;
0.034025,	-0.074983,	-0.711869,	0.080250,	0.306321
]
matrix added to itself:
[
0.960877,	-1.372046,	-1.089427,	-1.248804,	1.898643;
-0.246195,	-1.714530,	-0.682566,	1.661534,	0.189877;
0.068051,	-0.149966,	-1.423739,	0.160499,	0.612641
]


## Manual Comparison with Known Solution

To test the matrix multiplication, we manually define input matrices.
We also define the expected output, assuming we know the solution. Maybe we computed it by hand or with Matlab.
Again, we visually compare the expected output with the known solution simply by printing both.

In [8]:
%%writefile test.cu
#include "fmatrix.cuh"
#include "fmatrix_mult.cu"

int main() {
    // create two matrices on the host with given values
    fmatrix h_B = fmatrix_create_on_host(2,3);
    fmatrix h_C = fmatrix_create_on_host(3,2);
    // B = [ 2, 3, 4; 5, 6, 7 ]
    getfm(h_B,0,0) = 2;
    getfm(h_B,0,1) = 3;
    getfm(h_B,0,2) = 4;
    getfm(h_B,1,0) = 5;
    getfm(h_B,1,1) = 6;
    getfm(h_B,1,2) = 7;
    // C = [ -1, 2; -3, 4; -5, 6 ]
    getfm(h_C,0,0) = -1;
    getfm(h_C,0,1) = 2;
    getfm(h_C,1,0) = -3;
    getfm(h_C,1,1) = 4;
    getfm(h_C,2,0) = -5;
    getfm(h_C,2,1) = 6;
    // copy the matrices to the device
    fmatrix B = fmatrix_copy_to_device(h_B);
    fmatrix C = fmatrix_copy_to_device(h_C);
    printf("matrix B:\n");
    fmatrix_device_print(B);
    printf("matrix C:\n");
    fmatrix_device_print(C);
    // allocate matrix A
    fmatrix A = fmatrix_create_on_device(2,2);
    // compute A = 3.0*B*C
    fmatrix_mult(A,3.0,B,C);
    // print
    printf("matrix A=3*B*C:\n");
    fmatrix_device_print(A);
    fmatrix h_A_expected = fmatrix_create_on_host(2,2);
    // A_expected = [ -93, 120; -174, 228 ]
    getfm(h_A_expected,0,0) = -93;
    getfm(h_A_expected,0,1) = 120;
    getfm(h_A_expected,1,0) = -174;
    getfm(h_A_expected,1,1) = 228;
    printf("expected A:\n");
    fmatrix_print(h_A_expected);
}

Overwriting test.cu


In [ ]:
!nvcc -arch=sm_35 -Wno-deprecated-gpu-targets -g -G -I cuda-samples/Common/ -L/usr/local/cuda/include test.cu fmatrix.cu cuda_stuff.cu

In [ ]:
!./a.out

matrix B:
[
2.000000,	3.000000,	4.000000;
5.000000,	6.000000,	7.000000
]
matrix C:
[
-1.000000,	2.000000;
-3.000000,	4.000000;
-5.000000,	6.000000
]
matrix A=3*B*C:
[
-93.000000,	120.000000;
-174.000000,	228.000000
]
expected A:
[
-93.000000,	120.000000;
-174.000000,	228.000000
]


## Comparison with Known Code (Numpy)
For a test that is both easier to write and more reliable, let's use
Python/Numpy to create the matrices, compute the correct result, and
compare the result to the output of our code.

We exchange the matrices between Numpy and our program using CSV files.
First, we write our test program in C, which reads the input matrices
from CSV files and writes the result to another CSV file.

In [9]:
%%writefile test.cu
#include "fmatrix.cuh"
#include "fmatrix_mult.cu"

int main() {
    // read two matrices on the host from csv files
    fmatrix B = fmatrix_device_from_csv("test_B.csv");
    fmatrix C = fmatrix_device_from_csv("test_C.csv");
    // allocate matrix A
    fmatrix A = fmatrix_create_on_device(B.rows,C.cols);

    // compute A = 3.0*B*C
    fmatrix_mult(A,3.0,B,C);

    fmatrix_device_to_csv("test_A.csv",A);

    fmatrix_free_on_device(&B);
    fmatrix_free_on_device(&C);
    fmatrix_free_on_device(&A);
}

Overwriting test.cu


We need to compile our program so that we can call it from Python.

In [ ]:
!nvcc -arch=sm_35 -Wno-deprecated-gpu-targets -g -G -I cuda-samples/Common/ -L/usr/local/cuda/include test.cu fmatrix.cu cuda_stuff.cu

In [ ]:
!./a.out

Cannot open file.
a.out: fmatrix.cu:34: fmatrix fmatrix_create_on_host(int, int): Assertion `cols>0' failed.


Now comes the test program in Python. We create test inputs, save them
to CSV files, then call our test code, read its output from a CSV file.
Then we carry out the same computation in Numpy and compare its result
with our test output.

In [ ]:
import numpy as np
# create test inputs
B = np.array([[1,2,3],[4,5,6]])
C = np.array([[7,8],[9,10],[11,12]])
# and write them to a file
np.savetxt("test_B.csv",B,delimiter=',')
np.savetxt("test_C.csv",C,delimiter=',')
# call test code
!./a.out
# read the result of the test code
# Note: ndmin=2 since we want a 2D-matrix
#       (otherwise it can be flattend to a vector)
A_test = np.loadtxt("test_A.csv",delimiter=',',ndmin=2)
print("From C code:\n",A_test)
# compute the correct result
A = 3.0*B@C
print("From Numpy code:\n",A)
# compare correct result with test output
result = np.allclose(A,A_test)
print("Test passed?", result)

From C code:
 [[174. 192.]
 [417. 462.]]
From Numpy code:
 [[174. 192.]
 [417. 462.]]
Test passed? True


## Random Test Inputs (with Numpy)

How do we pick good test inputs? Did we think of all tricky situations (say, matrices that are column or row vectors)?
Instead of manually choosing test inputs, we can carry out a (potentially large) number of random tests.

In [ ]:
%%time
import numpy as np
from random import randint

nb_tests = 50
max_dimension = 5

for i in range(nb_tests):
  # create test inputs
  # choose random matrix dimensions
  n = randint(1,max_dimension)
  m = randint(1,max_dimension)
  k = randint(1,max_dimension)
  B = np.random.rand(n,k)
  C = np.random.rand(k,m)
  # and write them to a file
  np.savetxt("test_B.csv",B,delimiter=',')
  np.savetxt("test_C.csv",C,delimiter=',')
  # call test code
  !./a.out
  # read the result of the test code
  A_test = np.loadtxt("test_A.csv",delimiter=',',ndmin=2)
  # print("From C code:\n",A_test)
  # compute the correct result
  A = 3.0*B@C
  # print("From Numpy code:\n",A)
  # compare correct result with test output
  result = np.allclose(A,A_test)
  if not result:
    print("Wrong test result:")
    print("B:\n",B)
    print("C:\n",C)
    print("expected:\n",A)
    print("got wrong result:\n",A_test)
    break
if result:
  print("All random ",nb_tests," test passed.")



All random  50  test passed.
CPU times: user 593 ms, sys: 369 ms, total: 962 ms
Wall time: 47.1 s


# Debugging
We assume that you already compiled with debugging info on the host (`-g`) and device (`-G`).


Run the debugger cuda-gdb, stopping at the first error that is detected. Shows first the call stack on the GPU, the values of local variables, then the call stack on the host (thread 1).

In [ ]:
! printf "set cuda memcheck on\nset cuda api_failures stop\ncatch throw\nr\nbt\ninfo locals\nthread 1\nbt\n" > tmp.txt
! cat tmp.txt
! cuda-gdb -batch -x tmp.txt ./a.out

set cuda memcheck on
set cuda api_failures stop
catch throw
r
bt
info locals
thread 1
bt
Catchpoint 1 (throw)
[Thread debugging using libthread_db enabled]
Using host libthread_db library "/lib/x86_64-linux-gnu/libthread_db.so.1".
[New Thread 0x7f7d0a62c000 (LWP 1534)]
[Detaching after fork from child process 1535]
[New Thread 0x7f7d09aff000 (LWP 1540)]
[New Thread 0x7f7d092fe000 (LWP 1541)]
[Thread 0x7f7d09aff000 (LWP 1540) exited]
[Thread 0x7f7d0a62c000 (LWP 1534) exited]
[Thread 0x7f7d0cdce000 (LWP 1529) exited]
[Inferior 1 (process 1529) exited normally]
tmp.txt:5: Error in sourced command file:
No stack.


In [ ]:
!cuda-memcheck ./a.out

========= CUDA-MEMCHECK
========= This tool is deprecated and will be removed in a future release of the CUDA toolkit
========= Please use the compute-sanitizer tool as a drop-in replacement
========= ERROR SUMMARY: 0 errors
